# Install Dependencies

In [1]:
"""'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    🗿 READ ME 🗿
    - This Only Needs To Run Once.
    - This Needs to Restart the Kernel after Installing the Dependencies.  
    - To Avoid Unintended Restart: This adds an init.flag file in root folder after successful installation.
    - To Rerun: Delete init.flag file in root folder.
    
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''"""
# Import OS Dependency
import os, subprocess, sys

# Check If This Already Runned Once
if os.path.exists("init.flag"):
    print("Installation already completed.")
else:
    print("Starting Installation...")

    # Ensure pip is Installed
    subprocess.check_call([sys.executable, "-m", "ensurepip"], shell=True)

    # Install Main Dependencies
    subprocess.check_call(f'python -m pip install nltk "ffmpeg-python" "openai-whisper" pandas pytubefix bertopic "scikit-learn"', shell=True)
    
    # Install Other Dependencies
    subprocess.check_call(f'python -m pip install torch tqdm', shell=True)
    
    # Install BERTopic Dependencies
    subprocess.check_call(f'python -m pip install "numpy<2" "tf-keras"', shell=True)

    # Add Flag File to Set that this Already Runned Once
    open("init.flag", "w").close()

    # Restart the Kernel to Load Installed Dependences
    os._exit(00)

Installation already completed.


# Import Dependencies

In [2]:
# Import Main Dependencies
import os, re, nltk, ffmpeg, whisper
import pandas as pd
from pandas import DataFrame
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from pytubefix import YouTube, Stream
from pytubefix.cli import on_progress
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

# Import Other Dependencies
import torch
from tqdm.auto import tqdm

# Additional Downloads
nltk.download("punkt_tab")
nltk.download("averaged_perceptron_tagger_eng")
nltk.download("stopwords")

KeyboardInterrupt: 

# Define Utilities

In [3]:
def sanitize_filename(filename: str) -> str:
    # Escape Double Quotes
    filename = filename.replace('"', '\\"')

    # Replace Invalid Characters with "_"
    invalid_chars = re.compile(r'[<>:"/\\|?*]')
    sanitized_filename = invalid_chars.sub("_", filename)

    return sanitized_filename
def read_unique_items_from_file(file: str) -> list:
    with open(file, "r") as f:
        return list(set(url.strip() for url in f.readlines() if url.strip()))

# Set Configurations

In [6]:
# File Names
yt_video_links_filename = "YouTube Video Links.txt"
transcript_sentences_filename = "transcript_sentences.csv"
related_transcript_sentences_filename = "related_transcript_sentences.csv"
# Folder Names
video_output_path = "Video"
audio_output_path = "Audio"
transcription_output_path = "Transcription"
cities_path = "State Cities"
# Configs
remove_video = True
remove_audio = True
# Categories
presidential_candidates = {
    "Donald Trump": [
        "Donald", "Trump"
    ],
    "Kamala Harris": [
        "Kamala", "Harris"
    ]
}
state_cities = {
    "Michigan": read_unique_items_from_file(os.path.join(cities_path, "michigan-cities.txt")),
    "Arizona": read_unique_items_from_file(os.path.join(cities_path, "arizona-cities.txt")),
    "Pennsylvania": read_unique_items_from_file(os.path.join(cities_path, "pennsylvania-cities.txt"))
}
# Others
stop_words = set(stopwords.words("english"))
generic_abstract_nouns = {
    "thing", "stuff", "event",
    "aspect", "issue", "place",
    "person"
}
# Additional Preprocessing of Configurations
presidential_candidates = {presidential_candidate: list(set(names)) for presidential_candidate, names in presidential_candidates.items()}

{'Michigan': ['Port Huron',
  'Highland Park',
  'Flat Rock',
  'Mount Pleasant',
  'New Baltimore',
  'Ludington',
  'Livonia',
  'Byron Center',
  'Norton Shores',
  'Richmond',
  'Ironwood',
  'Oak Park',
  'River Rouge',
  'Rochester',
  'Whitmore Lake',
  'Zeeland',
  'Benton Harbor',
  'Davison',
  'Pontiac',
  'Taylor',
  'Wixom',
  'Haslett',
  'Farmington',
  'St. Clair Shores',
  'Northview',
  'Portage',
  'Bridgeport',
  'Allendale',
  'Flushing',
  'Garden City',
  'Grand Rapids',
  'Jackson',
  'East Grand Rapids',
  'Harper Woods',
  'Hastings',
  'Beverly Hills',
  'Hazel Park',
  'Westwood',
  'Three Rivers',
  'Grosse Pointe Park',
  'South Monroe',
  'Grand Haven',
  'St. Clair',
  'Ypsilanti',
  'Lansing',
  'Mason',
  'Albion',
  'Fraser',
  'South Lyon',
  'Clawson',
  'Midland',
  'Melvindale',
  'Jenison',
  'Trenton',
  'Ecorse',
  'Monroe',
  'Royal Oak',
  'Battle Creek',
  'Birmingham',
  'Hudsonville',
  'Southfield',
  'Fenton',
  'Troy',
  'Cadillac',
  '

# Collect Data (YouTube Videos)

In [ ]:
def download_youtube_video(video_filename: str, stream: Stream) -> tuple[str, str]:
    # Create Video Directory
    os.makedirs(video_output_path, exist_ok=True)
    
    # Set Path for Video File
    video_file = os.path.join(video_output_path, video_filename)
    
    # Delete Old Existing Video File (note: to clean any corrupted file)
    if os.path.exists(video_file):
        os.remove(video_file)
        
    # Download Video File
    print("") # Just New Line for Better Output
    print(f'Downloading (Video): {video_filename}')
    print("") # Just New Line for Better Output
    stream.download(output_path=video_output_path, filename=video_filename)
    print("") # Just New Line for Better Output
    print("") # Just New Line for Better Output
    
    # Return Video File and Name
    return video_file, video_filename

# Audio Extraction (Video to Audio)

In [ ]:
def extract_audio_from_video(video_file: str, video_filename: str) -> tuple[str, str]:
    # Create the Audio Directory
    os.makedirs(audio_output_path, exist_ok=True)

    # Set Audio File Name (".mp3")
    audio_filename = f'{os.path.splitext(video_filename)[0]}.mp3'

    # Set Path for Audio File
    audio_file = os.path.join(audio_output_path, audio_filename)
    
    # Delete Old Existing Audio File (note: to clean any corrupted file)
    if os.path.exists(audio_file):
        os.remove(audio_file)
    
    # Extract Audio File
    print(f'Extracting (Audio): {audio_filename}')
    print("") # Just New Line for Better Output
    (
        ffmpeg
        .input(video_file)
        .output(audio_file, format="mp3", acodec="libmp3lame", loglevel="info")
        .run(overwrite_output=True)
    )
    
    # Return Audio File and Name
    return audio_file, audio_filename

# Transcription (Audio to Text)

In [ ]:
def transcribe_audio_to_text(audio_file: str, audio_filename: str, index: int):
    # Create the Transcription Directory
    os.makedirs(transcription_output_path, exist_ok=True)
    
    # Set Transcription File Name (".txt")
    transcription_filename = f'{os.path.splitext(audio_filename)[0]}.txt'
    
    # Set Path for Transcription File
    transcription_file = os.path.join(transcription_output_path, transcription_filename)
            
    # Get/Download OpenAI Whisper Model
    """ 
    Models: 
        tiny, base, small, medium, large, turbo
    English-Only:
        tiny.en, base.en, small.en, medium.en
    
    Required VRAM:              Speed:
        1) 1GB - tiny, base         1) 10x - tiny
        2) 2GB - small              2) 8x - turbo
        3) 5GB - medium             3) 7x - base
        4) 6GB - turbo              4) 4x - small
        5) 10GB - large             5) 2x - medium
                                    6) 1x - large
    
    Quote from OpenAI: 
        - The .en models for English-only applications tend to perform better, especially for the tiny.en and base.en models.
        We observed that the difference becomes less significant for the small.en and medium.en models.
    
    Note: 4GB lang VRAM ko kaya small.en ginamit
    """  
    print(f'Transcribing (Text): {transcription_filename}')
    print("") # Just New Line for Better Output
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    model = whisper.load_model("tiny.en", device=device)

    # Transcribe Audio File (Saves Whole Text in Memory Before Disk to Avoid Corruption)
    result = model.transcribe(audio_file, fp16=False, verbose=False)
    with open(transcription_file, "w") as f:
        f.write(result["text"])

# Execute Data Gathering

In [7]:
yt_urls = read_unique_items_from_file(yt_video_links_filename)

with tqdm(total=len(yt_urls), desc="Getting YouTube URLs") as pbar:
    for index, url in enumerate(yt_urls):
        current = f'[{index+1}/{len(yt_urls)}]'
                
        # Get Video Information
        yt = YouTube(url, on_progress_callback=on_progress)
        stream = yt.streams.get_audio_only()

        # Sanitize Video File Name
        video_filename = sanitize_filename(stream.default_filename)
        
        # Get File Name Without Extension (e.g., ".mp4")
        transcription_filename = f'{os.path.splitext(video_filename)[0]}.txt'
        
        # Skip If Transcription Already Exists
        transcription_exists = False
        pbar.set_description(f'Checking Existing Transcription File {current}')
        if os.path.exists(transcription_output_path):
            for existing_transcription_filename in os.listdir(transcription_output_path):
                if existing_transcription_filename == transcription_filename:
                    existing_transcription_path = os.path.join(transcription_output_path, existing_transcription_filename)
                    if os.path.exists(existing_transcription_path):
                        transcription_exists = True
        if transcription_exists:
            pbar.update(1)
            continue

        # Download YouTube Video
        pbar.set_description(f'Downloading (Video) {current}')
        video_file, video_filename = download_youtube_video(video_filename, stream)
        
        # Extract Audio from Video -> Delete/Keep Video File
        pbar.set_description(f'Extracting (Audio) {current}')
        audio_file, audio_filename = extract_audio_from_video(video_file, video_filename)
        if remove_video: os.remove(video_file)
        
        # Transcribe Audio to Text -> Delete/Keep Audio File
        pbar.set_description(f'Transcribing (Text) {current}')
        transcribe_audio_to_text(audio_file, audio_filename, index)
        if remove_audio: os.remove(audio_file)
        
        pbar.update(1)
    pbar.set_description("Finished Data Gathering")

['https://www.youtube.com/watch?v=dxnMTg42bLA',
 'https://www.youtube.com/watch?v=V84s1Kb37WM',
 'https://www.youtube.com/watch?v=WwN0kHydiUc']

# Data Preprocessing

In [ ]:
def process_transcripts_into_csv_of_sentences() -> DataFrame:
    # Initialize List of Sentences
    list_of_sentences = []
    
    def is_sentence_complete(sentence: str) -> bool:
        """
        Its a Proper Sentence If:
            1) It has Atleast 1 Noun or Pronoun
            2) It has Atleast 1 Verb
        """
        pos_tags = pos_tag(word_tokenize(sentence)) # POS Tagging
        has_subject = any(tag in ["NN", "NNS", "NNP", "NNPS"] for _, tag in pos_tags) # Exclude Sentence w/out Noun and Pronoun
        has_verb = any(tag.startswith("VB") for _, tag in pos_tags) # Exclude Sentence w/out Verb
    
        return has_subject and has_verb
    
    # Collect List of All Sentences from Transcripts
    for filename in os.listdir(transcription_output_path):
        file_path = os.path.join(transcription_output_path, filename)
        
        with open(file_path, "r") as file:
            text = file.read()
            
            # Split into Sentences
            sentences = sent_tokenize(text)

            # Filter Proper Sentences (With Noun/Proper-Noun and Verb)
            sentences = [sentence for sentence in sentences if is_sentence_complete(sentence)]
            
            # Add Sentence to the List
            list_of_sentences.extend(sentences)

    # Save List of All Sentences into CSV file
    df = pd.DataFrame(list_of_sentences, columns=["Sentence"])
    df.to_csv(transcript_sentences_filename, index=False)
    return df

process_transcripts_into_csv_of_sentences().head()

# Text Cleaning: Topic Modeling and Sentence Filtering

In [ ]:
def filter_related_sentences() -> DataFrame:
    # Get All Sentences from Transcript
    df = pd.read_csv(transcript_sentences_filename)
    sentences = df["Sentence"].tolist()
    
    # Set Filter for Words as Possible Topics
    def filter_possible_topics(text) -> list:
        """
        Filter Words If its a Possible Topic:
            1) Only Nouns and Proper Nouns (e.g. Dollars, Currency)
            2) No Stop Words (e.g. in, to)
            3) No Generic Abstract Nouns (e.g. thing, stuff)
            4) Minumum of Three Letter Words (e.g. USA)
            5) Exclude Numbers
        """
        
        pos_tags = pos_tag(word_tokenize(text)) # POS Tagging
        possible_topics = [
            token.lower() for token, pos in pos_tags
            if pos in ["NN", "NNS", "NNP", "NNPS"] # Nouns / Proper Nouns
            and token.lower() not in stop_words # Exclude Stop Words
            and token.lower() not in generic_abstract_nouns # Exclude Generic Abstract Nouns
            and len(token) > 2 # Exclude One/Two Letter Words
            and not token.isnumeric() # Exclude Numbers
        ]
        
        return possible_topics
    vectorizer_model = CountVectorizer(tokenizer=filter_possible_topics)

    # Train BERTopic model
    topic_model = BERTopic(
        embedding_model="all-MiniLM-L6-v2",
        vectorizer_model=vectorizer_model
    )
    topic_model.fit_transform(sentences)
    
    # Get BERTopic Results
    topic_info = topic_model.get_topic_info()
    
    # Initialize Lists for our filtered results
    list_of_related_sentences = []
    
    # Analyze each topic row in topic_info
    for _, row in topic_info.iterrows():
        if row["Topic"] == -1: continue # Skip Outlier

        # Get List of Topics and its Related Sentences
        topic_keywords = row["Representation"]
        related_sentences = row["Representative_Docs"]
        
        # Check Candidate Mentions in Topics
        presidential_candidate_mentions = set() # Avoid Duplicates
        for presidential_candidate, names in presidential_candidates.items():
            if (
                any(name.lower() == keyword.lower() for name in names for keyword in topic_keywords) 
                or any(presidential_candidate.lower() == keyword.lower() for keyword in topic_keywords)
            ): 
                presidential_candidate_mentions.add(presidential_candidate)
        
        # Make Sure Only 1 Candidate is Mentioned
        if len(presidential_candidate_mentions) != 1: continue

        # Check State Mentions in Topics (Including Cities)
        state_mentions = set() # Avoid Duplicates
        for state, cities in state_cities.items():
            if (
                any(city.lower() == keyword.lower() for city in cities for keyword in topic_keywords) 
                or any(state.lower() == keyword.lower() for keyword in topic_keywords)
            ): 
                state_mentions.add(state)

        # Make Sure Only 1 State is Mentioned
        if len(presidential_candidate_mentions) != 1: continue
        
        """
        Add Related Sentences Only If:
            1) Only 1 Candidate is Mentioned
            2) Only 1 State is Mentioned
        """
        if len(presidential_candidate_mentions) == 1 and len(state_mentions) == 1:
            presidential_candidate = presidential_candidate_mentions[0]
            state = state_mentions[0]

            # Add All Related Sentences with Corresponding Presidential Candidate, State, and Topic Keywords
            for sentence in related_sentences:
                list_of_related_sentences.append({
                    "Sentence": sentence,
                    "Presidential_Candidate": presidential_candidate,
                    "State": state,
                    "Topic_Keywords": topic_keywords
                })
    
    # Save List of All Related Sentences into CSV file
    df = pd.DataFrame(list_of_related_sentences)
    df.to_csv(related_transcript_sentences_filename, index=False)
    return df

filter_related_sentences().head()

In [ ]:
def print_statistics():
    df = pd.read_csv(related_transcript_sentences_filename)
    
    # Group by candidate and state and count
    stats = df.groupby(["Presidential_Candidate", "State"]).size().reset_index(name="count")
    
    print("\nSentence counts per candidate and state:")
    print("----------------------------------------")
    for _, row in stats.iterrows():
        print(f'{row["Presidential_Candidate"]} - {row["State"]}: {row["count"]} sentences')

print_statistics()

# Data Sentiment Annotation

# Train-Validation-Test Split

# Training: Sentiment Analysis with BERT

# Validation: Hyperparameter Tuning and Model Optimization

# Testing: Model Evaluation

# Comparative Analysis 

# Proof of Concept